In [ ]:
import dask.dataframe as dd
import dask.multiprocessing
import dask.threaded
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [11]:
# Arquivos atuais
df_2005_2021 = dd.read_csv(['csv_2005_2020/inf_diario_fi_20*.csv'], sep=";", dtype={'TP_FUNDO': 'object'}) #44645554
df_2005_2021 = df_2005_2021.compute()
# cria a coluna vazia
df_2005_2021['TP_FUNDO'] = None
df_2005_2021 = dd.from_pandas(df_2005_2021, npartitions=4)

df_atual = dd.read_csv(['csv/inf_diario_fi_20*.csv'], sep=";", dtype={'TP_FUNDO': 'object'}) #17925211

df = dd.concat([df_2005_2021, df_atual]) 

print(df.columns)
print(f'O dataframe tem {len(df)} registros')

[########################################] | 100% Completed | 13.53 s
[########################################] | 100% Completed | 13.63 s
Index(['CNPJ_FUNDO', 'DT_COMPTC', 'VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ',
       'CAPTC_DIA', 'RESG_DIA', 'NR_COTST', 'TP_FUNDO'],
      dtype='object')
[########################################] | 100% Completed | 5.81 ss
[########################################] | 100% Completed | 5.74 s
O dataframe tem 62570765 registros


In [12]:
# transforma o campo DT_COMPTC
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('.', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('/', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.replace('-', '')
df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].str.zfill(14)

# cria o campo CO_PRD
df['CO_PRD'] = df['CNPJ_FUNDO']

# cria e formata o campo DT_REF, com a data de referência
print('Formatando campo DT_REF')
df.assign(DT_COMPTC=dd.to_datetime(df['DT_COMPTC'], format='%Y-%m-%d', errors='coerce'))
df['DT_REF'] = df['DT_COMPTC']

Formatando campo DT_REF


In [13]:
# faz o sort do df
df = df.sort_values(by=['CO_PRD', 'DT_REF'])

# Convert to a Pandas DataFrame because dask was being slow with the select logic below
print('Converte dataframe dask para pandas')
df = df.compute()

# cria uma nova coluna com o percentual de resgate para o dia
print('Calculando campo PC_RESG')
df['PC_RESG'] = df['RESG_DIA'] / df.groupby(['CO_PRD'])['VL_PATRIM_LIQ'].shift(1)

# Convert back to a Dask dataframe because we want that juicy parallelism
print('Converte dataframe pandas para dask')
df = dd.from_pandas(df, npartitions=4)

df['CNPJ_FUNDO'] = df['CNPJ_FUNDO'].astype(str).str.zfill(14)
df['CO_PRD'] = df['CO_PRD'].astype(str).str.zfill(14)

# removendo itens duplicados
print('Removendo itens duplicados')
df = df.drop_duplicates(subset=['CO_PRD', 'DT_REF'], keep='last')

df.tail()

[#                                       ] | 3% Completed | 30.73 sms

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


[########################################] | 100% Completed | 54.70 s
[########################################] | 100% Completed | 54.77 s
Converte dataframe dask para pandas
[########################################] | 100% Completed | 41.78 s
[########################################] | 100% Completed | 41.73 s
Calculando campo PC_RESG
Converte dataframe pandas para dask
Removendo itens duplicados
[########################################] | 100% Completed | 54.24 s
[########################################] | 100% Completed | 54.21 s


,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,TP_FUNDO,CO_PRD,DT_REF,PC_RESG
578707,97929213000134,2023-08-25,84412931.79,11.107729,84411205.20,0.0,0.0,2,FI,97929213000134,2023-08-25,0.0
578708,97929213000134,2023-08-28,84459070.59,11.113726,84456776.59,0.0,0.0,2,FI,97929213000134,2023-08-28,0.0
578709,97929213000134,2023-08-29,84695430.03,11.144754,84692568.40,0.0,0.0,2,FI,97929213000134,2023-08-29,0.0
578710,97929213000134,2023-08-30,84718705.72,11.147742,84715275.32,0.0,0.0,2,FI,97929213000134,2023-08-30,0.0
578711,97929213000134,2023-08-31,84559196.05,11.126677,84555196.78,0.0,0.0,2,FI,97929213000134,2023-08-31,0.0


In [14]:
print('Salvando base completa')
df.to_csv(filename='base_completa.csv', single_file=True, index=False)

Salvando base completa
[#################################       ] | 83% Completed | 55.35 ss
[#################################       ] | 83% Completed | 55.45 s


IsADirectoryError: [Errno 21] Is a directory: '/Users/rodrigo/projects/cvm_inf_diario/base_completa.csv'

In [6]:
#df = df.compute()
df_fundo = df[df['CO_PRD'] == '68623479000156']
df_fundo = df_fundo.sort_values(by=['DT_REF'])
df_fundo.head()

[########################################] | 100% Completed | 62.33 s


,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,TP_FUNDO,CO_PRD,DT_REF,PC_RESG
122026,68623479000156,2005-01-03,2.380346e+09,1.455841,2.379162e+09,13057686.32,8171806.44,118472,<NA>,68623479000156,2005-01-03,NaN
122027,68623479000156,2005-01-04,2.387009e+09,1.456861,2.386383e+09,10184717.40,4631003.49,118638,<NA>,68623479000156,2005-01-04,0.001946
122028,68623479000156,2005-01-05,2.393090e+09,1.457766,2.392315e+09,9226515.84,4776444.60,118837,<NA>,68623479000156,2005-01-05,0.002002
122029,68623479000156,2005-01-06,2.397874e+09,1.458677,2.396945e+09,9331765.42,6196972.82,119015,<NA>,68623479000156,2005-01-06,0.002590
122030,68623479000156,2005-01-07,2.402132e+09,1.459528,2.401137e+09,8845482.36,6052447.41,119201,<NA>,68623479000156,2005-01-07,0.002525


In [7]:
df_fundo.tail()

[########################################] | 100% Completed | 63.82 s


,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,TP_FUNDO,CO_PRD,DT_REF,PC_RESG
410818,68623479000156,2024-01-17,6.256222e+08,7.464607,6.254870e+08,284024.96,1308381.85,29447,FI,68623479000156,2024-01-17,0.002089
410819,68623479000156,2024-01-18,6.257034e+08,7.467652,6.255289e+08,337432.34,550647.95,29449,FI,68623479000156,2024-01-18,0.000880
410820,68623479000156,2024-01-19,6.253565e+08,7.470566,6.251423e+08,161526.43,792198.36,29442,FI,68623479000156,2024-01-19,0.001266
410821,68623479000156,2024-01-22,6.249635e+08,7.473419,6.247257e+08,273264.64,928634.72,29445,FI,68623479000156,2024-01-22,0.001485
410822,68623479000156,2024-01-23,6.246430e+08,7.476582,6.245527e+08,195357.96,632714.83,29444,FI,68623479000156,2024-01-23,0.001013


In [8]:
df_fundo = df[df['CO_PRD'] == '03737223000124']
df_fundo = df_fundo.sort_values(by=['DT_REF'])
df_fundo.head()

[########################################] | 100% Completed | 64.60 s


,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,TP_FUNDO,CO_PRD,DT_REF,PC_RESG
40797,03737223000124,2005-01-03,7.500712e+08,1.996119,7.499489e+08,0.00,12595782.63,1,<NA>,03737223000124,2005-01-03,NaN
40798,03737223000124,2005-01-04,7.497514e+08,1.997613,7.496480e+08,0.00,862261.25,1,<NA>,03737223000124,2005-01-04,0.001150
40799,03737223000124,2005-01-05,7.500570e+08,1.998865,7.499035e+08,0.00,214277.94,1,<NA>,03737223000124,2005-01-05,0.000286
40800,03737223000124,2005-01-06,7.504768e+08,2.000091,7.502778e+08,0.00,85676.08,1,<NA>,03737223000124,2005-01-06,0.000114
40801,03737223000124,2005-01-07,7.627197e+08,2.001580,7.626416e+08,11805264.19,0.00,1,<NA>,03737223000124,2005-01-07,0.000000


In [9]:
df_fundo.tail()

[########################################] | 100% Completed | 63.36 s


,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,TP_FUNDO,CO_PRD,DT_REF,PC_RESG
9635,03737223000124,2024-01-17,2.140155e+08,15.044428,2.140025e+08,0.00,761020.45,1,FI,03737223000124,2024-01-17,3.543946e-03
9636,03737223000124,2024-01-18,2.138992e+08,15.057846,2.139615e+08,0.00,231895.85,1,FI,03737223000124,2024-01-18,1.083613e-03
9637,03737223000124,2024-01-19,2.142901e+08,15.068158,2.143232e+08,215241.12,105.05,1,FI,03737223000124,2024-01-19,4.909762e-07
9638,03737223000124,2024-01-22,2.152154e+08,15.057305,2.150993e+08,930483.03,0.00,1,FI,03737223000124,2024-01-22,0.000000e+00
9639,03737223000124,2024-01-23,2.158853e+08,15.072298,2.159514e+08,637983.74,17.92,1,FI,03737223000124,2024-01-23,8.331036e-08
